# Database

In this file the dataset is created in which the graph data will be stored. First part is to read the *.dem* files of the matches played on inferno, than convert the datas into ***HeteroData*** structure so that it can be stored in a PyG database.

## Creating HeteroData objects from game snapshots

In [1]:
import torch
from torch_geometric.data import HeteroData, DataLoader
from torch_geometric.data import Dataset, Data

import matplotlib.pyplot as plt
from math import floor
import pandas as pd
import numpy as np
import os

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)

Reading the inferno csv files

In [2]:
def get_inferno_matches(folder_path):
    matches = []
    for filename in os.listdir(folder_path + '/frames/'):
        if 'inferno' in filename:
            matches.append(filename)
    return matches

Reading dataframes.

In [3]:
def get_base_dataframes(folder_path, filename):
    playerFrames = pd.read_csv(folder_path + '/playerFrames/' + filename)
    kills = pd.read_csv(folder_path +'/kills/' + filename)
    rounds = pd.read_csv(folder_path +'/rounds/' + filename)


    rounds = rounds[['roundNum', 'tScore', "ctScore" ,'endTScore', 'endCTScore']]
    pf = playerFrames[['tick', 'roundNum', 'seconds', 'side', 'name', 'x', 'y', 'z','eyeX', 'eyeY', 'eyeZ', 'velocityX', 'velocityY', 'velocityZ',
        'hp', 'armor', 'activeWeapon','flashGrenades', 'smokeGrenades', 'heGrenades', 'totalUtility','isAlive', 'isReloading', 'isDefusing', 'isPlanting', 'isUnknown',
        'equipmentValue', 'equipmentValueRoundStart', 'hasHelmet','hasDefuse', 'hasBomb']]
    
    return pf, kills, rounds

Formatting the dataframes.

In [4]:
def format_base_dataframes(pf, kills, rounds):
    
    # Merge playerFrames with rounds
    pf = pf.merge(rounds, on='roundNum')
    
    # Calculate whether or not a player won the round
    pf["winsRounds"] = pf.apply(lambda x: 
        1 if ( (x['side'] == 'T') and (x['endTScore'] > x['tScore']) ) or ( (x['side'] == 'CT') and (x['endCTScore'] > x['ctScore']) )
        else 0, axis=1)

    pf['kills'] = 0
    pf['deaths'] = 0
    
    # Setting kill-counts
    for index, row in kills.iterrows():
        pf.loc[(pf['tick'] >= row['tick']) & (pf['name'] == row['attackerName']), 'kills'] += 1
        pf.loc[(pf['tick'] >= row['tick']) & (pf['name'] == row['victimName']), 'deaths'] += 1
        
    # Rounded-down seconds
    pf['sec'] = pf['seconds'].apply(lambda x: floor(x))
        
    return pf, kills, rounds

In [5]:
def get_dummies(pf):
    
    # Tracked weapons
    tracked_weapons = ['activeWeapon_AK-47','activeWeapon_Knife','activeWeapon_AWP','activeWeapon_M4A1','activeWeapon_Smoke Grenade','activeWeapon_M4A4','activeWeapon_Galil AR','activeWeapon_Desert Eagle','activeWeapon_Flashbang','activeWeapon_Glock-18','activeWeapon_USP-S']
    
    # Create dummie cols
    dummies = pd.get_dummies(pf['activeWeapon'], prefix="activeWeapon",drop_first=False)
    for col in dummies.columns:
        if col not in tracked_weapons:
            dummies = dummies.drop(col, axis=1)
    
    dummies = dummies*1
    pf = pf.merge(dummies, left_index = True, right_index = True, how = 'left')
    
    return pf

In [6]:
def create_players_dataframes(pf):
    
    startAsCTPlayerNames = pf[(pf['side'] == 'CT') & (pf['roundNum'] == 1)]['name'].unique()
    startAsTPlayerNames = pf[(pf['side'] == 'T') & (pf['roundNum'] == 1)]['name'].unique()
    players = {}

    # Team 1: start on CT side
    players[0] = pf[pf['name'] == startAsCTPlayerNames[0]].drop_duplicates(subset=['sec','roundNum']).copy()
    players[1] = pf[pf['name'] == startAsCTPlayerNames[1]].drop_duplicates(subset=['sec','roundNum']).copy()
    players[2] = pf[pf['name'] == startAsCTPlayerNames[2]].drop_duplicates(subset=['sec','roundNum']).copy()
    players[3] = pf[pf['name'] == startAsCTPlayerNames[3]].drop_duplicates(subset=['sec','roundNum']).copy()
    players[4] = pf[pf['name'] == startAsCTPlayerNames[4]].drop_duplicates(subset=['sec','roundNum']).copy()

    # Team 2: start on T side
    players[5] = pf[pf['name'] == startAsTPlayerNames[0]].drop_duplicates(subset=['sec','roundNum']).copy()
    players[6] = pf[pf['name'] == startAsTPlayerNames[1]].drop_duplicates(subset=['sec','roundNum']).copy()
    players[7] = pf[pf['name'] == startAsTPlayerNames[2]].drop_duplicates(subset=['sec','roundNum']).copy()
    players[8] = pf[pf['name'] == startAsTPlayerNames[3]].drop_duplicates(subset=['sec','roundNum']).copy()
    players[9] = pf[pf['name'] == startAsTPlayerNames[4]].drop_duplicates(subset=['sec','roundNum']).copy()
    
    return players


In [7]:
# Calculate closest map-graph node
def find_closest(row):
    nodes = pd.read_csv('../inferno_graph/graph_models/manual/nodes_v1_5.csv')
    distances = np.sqrt((nodes['x'] - row['x'])**2 + (nodes['y'] - row['y'])**2)
    return nodes.loc[distances.idxmin(), 'nodeId']

def calculate_closest_map_node_to_player(pf):
    
    # Read nodes and create players
    players = create_players_dataframes(pf)

    for idx, player in enumerate(players):
        
        players[idx] = players[idx].replace(True, 1)
        players[idx] = players[idx].replace(False, 0)
        players[idx]['isCT'] = players[idx]['side'].apply(lambda x: 1 if x == 'CT' else 0)
        players[idx]['closestId'] = players[idx].apply(find_closest, axis=1)

        del players[idx]['side']
        del players[idx]['activeWeapon']
        del players[idx]['winsRounds']
    
    return players

In [8]:
def get_player_edges(players):
    
    playerEdges = None

    for idx in range(0,len(players)):
        temp = players[idx][['roundNum','sec','closestId']].copy()
        temp['playerId'] = idx
        if playerEdges is None:
            playerEdges = temp
        else:
            playerEdges = pd.concat([playerEdges, temp])
    
    return playerEdges

In [9]:
def get_graph_level_data(players, rounds):
    # Copy players object
    graph_players = {}
    for idx in range(0,len(players)):
        graph_players[idx] = players[idx].copy()

    colsNotToRename = ['tick', 'roundNum', 'seconds', 'sec']

    # Rename columns except for tick, roundNum, seconds, floorSec
    for idx in range(0,len(graph_players)):
        
        for col in graph_players[idx].columns:
            if col not in colsNotToRename:
                graph_players[idx].rename(columns={col: "player" + str(idx) + "_" + col}, inplace=True)

    # Create a graph dataframe to store all players in 1 row per second
    graph_data = graph_players[0].copy()

    # Merge dataframes
    for i in range(1, len(graph_players)):
        graph_data = graph_data.merge(graph_players[i], on=colsNotToRename)
        
    graph_data = graph_data.merge(rounds, on=['roundNum'])
    graph_data['CTwinsRound'] = graph_data.apply(lambda x: 
        1 if 
        (x['endCTScore'] > x['ctScore'])
        else 0, axis=1)
    graph_data['team1AliveNum'] = graph_data[['player0_isAlive','player1_isAlive','player2_isAlive','player3_isAlive','player4_isAlive']].sum(axis=1)
    graph_data['team2AliveNum'] = graph_data[['player5_isAlive','player6_isAlive','player7_isAlive','player8_isAlive','player9_isAlive']].sum(axis=1)
    graph_data = graph_data[['roundNum','seconds','sec','team1AliveNum','team2AliveNum','CTwinsRound']]
    
    return graph_data

In [10]:
def get_nodes_and_edges():
    nodes = pd.read_csv('../inferno_graph/graph_models/manual/nodes_v1_5.csv')
    edges = pd.read_csv('../inferno_graph/graph_models/manual/edges_v1_5.csv')
    
    nodes['x'] = nodes['x'].astype('float32')
    nodes['y'] = nodes['y'].astype('float32')
    nodes['isPath'] = nodes['type'].apply(lambda x: 1 if x == 'path' else 0)
    
    return nodes, edges

In [11]:
def get_player_overall_statistics(players):
    # Needed columns
    neededC_stats = ['player_name', 'rating_2.0', 'DPR', 'KAST', 'Impact', 'ADR', 'KPR','total_kills', 'HS%', 'total_deaths', 'KD_ratio', 'dmgPR',
       'grenade_dmgPR', 'maps_played', 'saved_by_teammatePR', 'saved_teammatesPR','opening_kill_rating', 'team_W%_after_opening',
       'opening_kill_in_W_rounds', 'rating_1.0_all_Career', 'clutches_won_1on4', 'clutches_won_1on5']
    
    stats = pd.read_csv('../scrape-parse/hltv-stat/stats.csv')
    stats['player_name'] = stats['player_name'].apply(lambda x: x.replace('-', '') if '-' in x else x)
    stats['player_name'] = stats['player_name'].apply(lambda x: x.replace(' ', '') if '-' in x else x)
    stats = stats[neededC_stats]
    stats['rating_1.0_all_Career'] = stats['rating_1.0_all_Career'].str[:-7].astype('float32')
    
    inf_stats = pd.read_csv('../scrape-parse/hltv-stat/stats_inf.csv')
    inf_stats['player_name'] = inf_stats['player_name'].apply(lambda x: x.replace('-', '') if '-' in x else x)
    inf_stats['player_name'] = inf_stats['player_name'].apply(lambda x: x.replace(' ', '') if '-' in x else x)
    inf_stats = inf_stats[neededC_stats]
    inf_stats['rating_1.0_all_Career'] = inf_stats['rating_1.0_all_Career'].str[:-7].astype('float32')
    

    for col in stats.columns:
        if col != 'player_name':
            stats[col] = stats[col].astype('float32')
            stats.rename(columns={col: "overall_" + col}, inplace=True)
            
            inf_stats[col] = inf_stats[col].astype('float32')
            inf_stats.rename(columns={col: "inf_" + col}, inplace=True)
    
    for idx in range(0,len(players)):
        players[idx] = pd.merge(players[idx], stats, left_on='name', right_on='player_name', how='left').drop(columns=['player_name'])
        players[idx] = pd.merge(players[idx], inf_stats, left_on='name', right_on='player_name', how='left').drop(columns=['player_name'])
        
    return players

In [12]:
def create_playerFrameData_for_graph(players, roundNum, sec):
    playerFrameData = pd.DataFrame(columns=players[0].columns)
    for idx in range(0,len(players)):
        playerFrameData.loc[idx] = players[idx].loc[(players[idx]['roundNum'] == roundNum) & (players[idx]['sec'] == sec)].iloc[0]
        
    playerFrameData.drop(columns=['tick', 'roundNum', 'seconds', 'name', 'closestId'], inplace=True)
    playerFrameData = playerFrameData.astype('float32')
    
    return playerFrameData

In [13]:
def create_HeteroData(playerFrameData, playerEdges, graph_data, nodes, edges, roundNum, sec):
    data = HeteroData()

    # Create a sample heterogeneous graph with node, edge, and multiple graph-level features
    data['player'].x = torch.tensor(playerFrameData.astype('float32').values)
    data['map'].x = torch.tensor(nodes[['x','y','isPath']].astype('float32').values)

    data['map', 'connected_to', 'map'].edge_index = torch.tensor(edges.T.values)
    data['player', 'closest_to', 'map'].edge_index = torch.tensor(playerEdges.loc[ (playerEdges['roundNum'] == roundNum) & (playerEdges['sec'] == sec)][['playerId','closestId']].T.values)


    # Define multiple graph-level features
    data.y = {
        'roundNum': roundNum.astype('float32'),
        'sec': sec.astype('float32'),
        'team1AliveNum': torch.tensor(graph_data.loc[ (graph_data['roundNum'] == roundNum) & (graph_data['sec'] == sec)]['team1AliveNum'].iloc[0].astype('float32')),
        'team2AliveNum': torch.tensor(graph_data.loc[ (graph_data['roundNum'] == roundNum) & (graph_data['sec'] == sec)]['team2AliveNum'].iloc[0].astype('float32')),
        'CTwinsRound': torch.tensor(graph_data.loc[ (graph_data['roundNum'] == roundNum) & (graph_data['sec'] == sec)]['CTwinsRound'].iloc[0])
    }
    
    return data

Code to create the HeteroData object.

```python

## Dataset for HeteroData objects

In [14]:
class InfernoDataset(Dataset):
    def __init__(self, data_list):
        super(InfernoDataset, self).__init__()
        self.data_list = data_list

    def len(self):
        return len(self.data_list)

    def get(self, idx):
        return self.data_list[idx]


Codes testing the created dataset class

In [15]:
# infdataset = InfernoDataset([data])

# torch.save(infdataset, './data/_/test_dataset.pt')

# dataset = torch.load('./data/_/test_dataset.pt')

# loader = DataLoader(infdataset, batch_size=1, shuffle=True)

## Creating the dataset

In [15]:
folder_path = '../scrape-parse/demo/parse/2023'
matches = get_inferno_matches(folder_path)
hetero_data_list = []
skip_done = False
last_saved = ''
with open('last-saved.txt', 'r') as file:
    last_saved = file.read()

for idx,match in enumerate(matches):
        
    if last_saved == '':
        skip_done = True
    elif last_saved != match and skip_done == False:
        continue
    elif last_saved == match and skip_done == False:
        skip_done = True
        continue
    
    if match == 'blast-premier-spring-final-2023-g2-vs-cloud9-bo3-l6_WROLL2MODdQ0NexGWUJg2-vs-cloud9-m2-inferno.dem.csv'  or \
       match == 'iem-cologne-2023-g2-vs-faze-bo3-2rs0vwSX-MC7GnSL4ITW06g2-vs-faze-m1-inferno.dem.csv' or \
       match == 'iem-cologne-2023-g2-vs-astralis-bo3-_w4eg3Jhwy1dhctFRgGSl7g2-vs-astralis-m2-inferno.dem.csv' or \
       match == 'iem-katowice-2023-faze-vs-ihc-bo3-xzdrX8BWDhgJKqFJblorpafaze-vs-ihc-m1-inferno.dem.csv':
        continue
    
    pf, kills, rounds = get_base_dataframes(folder_path, match)
    pf, kills, rounds = format_base_dataframes(pf, kills, rounds)
    pf = get_dummies(pf)
    players = calculate_closest_map_node_to_player(pf)
    players = get_player_overall_statistics(players)
    player_edges = get_player_edges(players)
    graph_data = get_graph_level_data(players, rounds)
    nodes, edges = get_nodes_and_edges()

    rounds_list = pf['roundNum'].unique()
    for roundNum in rounds_list:
        sec_list = pf[pf['roundNum'] == roundNum]['sec'].unique()
        for sec in sec_list:
            playerFrameData = create_playerFrameData_for_graph(players, roundNum, sec)
            data = create_HeteroData(playerFrameData, player_edges, graph_data, nodes, edges, roundNum, sec)
            hetero_data_list.append(data)
        print(match,roundNum)
    
    if idx % 5 == 0:
        if os.path.exists('./data/inferno_graph_dataset_3.pt') == False:
            infdataset = InfernoDataset(hetero_data_list)
            torch.save(infdataset, './data/inferno_graph_dataset_3.pt')
            hetero_data_list = []
        elif os.path.exists('./data/inferno_graph_dataset_3.pt') == True:
            infdataset = torch.load('./data/inferno_graph_dataset_3.pt')
            infdataset.data_list = infdataset.data_list + hetero_data_list
            torch.save(infdataset, './data/inferno_graph_dataset_3.pt')
            hetero_data_list = []
        
        with open('last-saved.txt', 'w') as file:
            file.write(match)

iem-dallas-2023-mouz-vs-og-bo3-tGd7zF8hss-uT_IOarRz-8mouz-vs-og-m2-inferno.dem.csv 1
iem-dallas-2023-mouz-vs-og-bo3-tGd7zF8hss-uT_IOarRz-8mouz-vs-og-m2-inferno.dem.csv 2
iem-dallas-2023-mouz-vs-og-bo3-tGd7zF8hss-uT_IOarRz-8mouz-vs-og-m2-inferno.dem.csv 3
iem-dallas-2023-mouz-vs-og-bo3-tGd7zF8hss-uT_IOarRz-8mouz-vs-og-m2-inferno.dem.csv 4
iem-dallas-2023-mouz-vs-og-bo3-tGd7zF8hss-uT_IOarRz-8mouz-vs-og-m2-inferno.dem.csv 5
iem-dallas-2023-mouz-vs-og-bo3-tGd7zF8hss-uT_IOarRz-8mouz-vs-og-m2-inferno.dem.csv 6
iem-dallas-2023-mouz-vs-og-bo3-tGd7zF8hss-uT_IOarRz-8mouz-vs-og-m2-inferno.dem.csv 7
iem-dallas-2023-mouz-vs-og-bo3-tGd7zF8hss-uT_IOarRz-8mouz-vs-og-m2-inferno.dem.csv 8
iem-dallas-2023-mouz-vs-og-bo3-tGd7zF8hss-uT_IOarRz-8mouz-vs-og-m2-inferno.dem.csv 9
iem-dallas-2023-mouz-vs-og-bo3-tGd7zF8hss-uT_IOarRz-8mouz-vs-og-m2-inferno.dem.csv 10
iem-dallas-2023-mouz-vs-og-bo3-tGd7zF8hss-uT_IOarRz-8mouz-vs-og-m2-inferno.dem.csv 11
iem-dallas-2023-mouz-vs-og-bo3-tGd7zF8hss-uT_IOarRz-8mouz-vs-og

In [16]:
# try...
# catch
# IndexError                                Traceback (most recent call last)
# d:\BME\csgo\csgo-gnn\model\database.ipynb Cell 25 line 3
#      35 sec_list = pf[pf['roundNum'] == roundNum]['sec'].unique()
#      36 for sec in sec_list:
# ---> 37     playerFrameData = create_playerFrameData_for_graph(players, roundNum, sec)
#      38     data = create_HeteroData(playerFrameData, player_edges, graph_data, nodes, edges, roundNum, sec)
#      39     hetero_data_list.append(data)

# d:\BME\csgo\csgo-gnn\model\database.ipynb Cell 25 line 4
#       2 playerFrameData = pd.DataFrame(columns=players[0].columns)
#       3 for idx in range(0,len(players)):
# ----> 4     playerFrameData.loc[idx] = players[idx].loc[(players[idx]['roundNum'] == roundNum) & (players[idx]['sec'] == sec)].iloc[0]
#       6 playerFrameData.drop(columns=['tick', 'roundNum', 'seconds', 'name', 'closestId'], inplace=True)
#       7 playerFrameData = playerFrameData.astype('float32')

# File ~\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pandas\core\indexing.py:1103, in _LocationIndexer.__getitem__(self, key)
#    1100 axis = self.axis or 0
#    1102 maybe_callable = com.apply_if_callable(key, self.obj)
# -> 1103 return self._getitem_axis(maybe_callable, axis=axis)

# File ~\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pandas\core\indexing.py:1656, in _iLocIndexer._getitem_axis(self, key, axis)
#    1653     raise TypeError("Cannot index by location index with a non-integer key")
#    1655 # validate the location
# -> 1656 self._validate_integer(key, axis)
# ...
#    1587 len_axis = len(self.obj._get_axis(axis))
#    1588 if key >= len_axis or key < -len_axis:
# -> 1589     raise IndexError("single positional indexer is out-of-bounds")

# IndexError: single positional indexer is out-of-bounds
